MS_bulletin_tex_labeling3.py

# Import libraries

In [44]:
from collections import defaultdict
import json, re, pickle, nltk
import copy
import os
import codecs
import numpy as np 
import pandas as pd

# Define regex for tagging

In [45]:
def secondary_tagger(T):
    """
    regexs for tagging
    """
    
    # for version tagging regexs
    C=[r'^[0-9]+(\.|x)+[0-9a-zA-Z\-\.]{1,}$' ,r'^[0-9.x]{2,}\.+-[0-9a-zA-Z.]+$',r'^[0-9\.x]+\.?[a-zA-Z.]+$',r'^[0-9\.x]+_[a-zA-Z0-9.]+$',r'^[0-9\.x]+\%[0-9a-zA-Z.]+$',r'^[0-9\.x]+-([0-9.]+[a-zA-Z0-9.\-_]*|[a-zA-Z0-9.\-_]*[0-9.]+)$',r'^v[0-9x\.\-]{2,}$']

    # Lists for cwe relevant term tagging
    ones=[("MITM"),('(csrf)',), ('(xss)',), ('..',), (u'CSRF',), (u'HTML',), (u'Integer',) ,("sql",) , (u'Race',), (u'Use-after-free',), (u'XSS',), (u'access',), (u'account',), (u'allows',), (u'application',), (u'arbitrary',), ('attack',), (u'attackers',), ('authenticate',), (u'authenticated',), ('authentication',), ('authorization',), (u'blocked',), (u'brute-force',), (u'bypass',), (u'cleartext',), (u'code',), (u'commands',), ('configuration',), (u'crash',), (u'credentials',), ('csrf',), ('ddos',), ('denial',), ('denial of service',), ('distributed denial of service',), ('dos',), (u'execute',), ('exposure',), (u'file.',), (u'files',), ('fingerprinting',), (u'forgery',), (u'gain',), (u'handlers',), (u'hijack',), (u'hook',), (u'hook-handler',), (u'information',), (u'inject',), (u'kernel',), (u'kernel-mode',), (u'local',), (u'malware',), ('man-in-the-middle',), (u'memory',), (u'metacharacters',), (u'obtain',), (u'obtained',), ('overflow',), ('overflows',), (u'overwrite',), (u'password',), ('permissions',), ('plaintext',), (u'privileges',), (u'remote',), (u'requests',), (u'script',), (u'scripting',), (u'sensitive',), (u'service',), ('sign',), (u'signature-based',), ('signedness',), (u'symlink',), ('truncation',), ('underflow',), ('underflows',), (u'user-space',), (u'value',), ('xss',)]
    twos=[('.', '.'), ("integer","underflow"),("Integer", "underflows"),(u'Race', u'condition'), (u'SQL', u'commands'), ('SQL', 'injection'), (u'Use-after-free', u'vulnerability'), ('access', 'control'), (u'access', u'restrictions'), (u'application', u'crash'), (u'arbitrary', u'SQL'), ('arbitrary', 'certificate'), ('arbitrary', 'code'), (u'arbitrary', u'files'), ('arbitrary', 'password'), ('authentication', 'issues'), ('buffer', 'error'), ('buffer', 'overflow'), (u'bypass', u'authentication'), ('code', 'injection'), ('credentials', 'management'), ('cross', 'site forgery'), ('cross-site', 'forgery'), ('cross-site', 'request'), ('cross-site', 'scripting'), ('cryptographic', 'issues'), ('design', 'error'), ('design', 'errors'), ('dot', 'dot'), ('finger', 'printing'), ('format', 'string'), ('gain', 'privileges'), ('hard', 'links'), ('infinite', 'loop'), ('infinite', 'loops'), ('information', 'disclosure'), ('information', 'leak'), ('inject', 'code'), (u'injection', u'vulnerability'), ('input', 'validation'), ('integer', 'overflow'), ('key', 'disclosure'), ('link', 'following'), (u'local', u'users'), ('memory', 'exhaustion'), ('memory', 'leak'), ('memory', 'leaks'), ('modify', 'query'), ('numeric', 'error'), ('numeric', 'errors'), ('path', 'traversal'), (u'potentially', u'sensitive'), ('race', 'conditions'), ('remote', 'attacker'), (u'remote', u'attackers'), ('replay', 'attack'), ('resource', 'management'), (u'sensitive', u'information'), ('sql', 'injection'), ('symbolic', 'links'), (u'symlink', u'attack'), (u'traversal', u'vulnerability'), ('weak', 'key'), (u'web', u'script')]
    threes=[(u'Directory', u'traversal', u'vulnerability'), (u'Multiple', u'cross-site', u'request'), (u'SQL', u'injection', u'vulnerabilities'), (u'SQL', u'injection', u'vulnerability'), (u'arbitrary', u'PHP', u'code'), (u'arbitrary', u'SQL', u'commands'), (u'arbitrary', u'web', u'script'), ('buffer', 'boundary', 'error'), ('cross', 'site', 'request'), ('cross', 'site', 'scripting'), ('cross-site', 'request', 'forgery'), (u'denial', u'of', u'service'), (u'execute', u'arbitrary', u'SQL'), ('execute', 'arbitrary', 'code'), (u'execute', u'arbitrary', u'commands'), (u'format', u'string', u'specifiers'), ('format', 'string', 'vulnerability'), ('gain', 'administrative', 'access'), (u'hijack', u'the', u'authentication'), ('inject', 'arbitrary', 'code'), ('modify', 'query', 'logic'), (u'obtain', u'sensitive', u'information'), ('os', 'command', 'injection'), (u'remote', u'authenticated', u'users'), ('resource', 'management', 'error'), ('resource', 'management', 'errors')]


    for j in range(len(T)):
        # version tagging:
        for regex in C:
            s,tag=T[j]
            if tag =="O" and re.search(regex,s):
                T[j]=(T[j][0],"B:version")
            #if j>0 and re.search(r'[0-9x\.\-]',T[j][0] ):
            if j>0 and re.search(r'([0-9x\-]){1,}|([0-9x\.\-]){2,}',T[j][0] ):
                if (T[j-1][0]=="before" or T[j-1][0]=="through") and not re.search(r'(\w[a-z\-]){2,}',T[j][0]):
                    if j>1 and re.search("version", T[j-2][1]):
                        T[j-1]=(T[j-1][0],"I:version")
                        T[j]=(T[j][0], "I:version")
                    else: 
                        T[j-1]=(T[j-1][0],"B:version")
                        T[j]=(T[j][0], "I:version")
                    if re.search("application", T[j-1][1]):
                        T[j]=(T[j][0], "B:version")
            if len(T)>j+1 and re.search(r'[0-9x\.\-]',T[j][0] ):
                if (T[j+1][0]=="and" or T[j+1][0]=="or") and T[j+2][0]=="earlier":
                    T[j]=(T[j][0], "B:version")
                    T[j+1]=(T[j+1][0],"I:version")
                    T[j+2]=(T[j+2][0], "I:version")
                if T[j+1][0]=="and" and T[j+2][0]=="below":
                    T[j]=(T[j][0], "B:version")
                    T[j+1]=(T[j+1][0],"I:version")
                    T[j+2]=(T[j+2][0], "I:version")

            if re.search("version", T[j][1]) and len(T)>j+2 and re.search(r'^[,]{1}$|^and$',T[j+1][0]) and re.search(r'^[0-9]',T[j+2][0]):
                T[j+2]=(T[j+2][0],"B:version")
            if re.search("version", T[j][1]) and len(T)>j+3 and T[j+1][0]=="," and T[j+2][0]=="and" and re.search(r'^[0-9]',T[j+3][0]):
                T[j+3]=(T[j+3][0],"B:version")
            if T[j][0]=="all" and len(T)>j+2 and T[j+1][0]=="supported" and (T[j+2][0]=="versions" or T[j+2][0]=="releases"):
                T[j] = (T[j][0],"B:version")
                T[j+1] = (T[j+1][0],"B:version")
                T[j+2] = (T[j+2][0],"B:version")
                

        # update tagging:
        if re.search(r'^[A-Z]{1,3}[0-9]$', T[j][0]) and T[j][1]=="O":
            A=surroundings(T, j, 3)[0:3]
            for s,tag in A:
                if tag=="B:version":
                    T[j]=(T[j][0],'B:update')
        if  re.search(r'^[0-9a-z\-_.]*\%[0-9a-z\-_.]+', T[j][0]) and T[j][1]=="O":
            T[j]=(T[j][0],'B:update')
        if re.search(r'^-[a-zA-Z0-9.]+$', T[j][0]) and T[j][1]=="O":
            T[j]=(T[j][0],'B:update')
        if re.search(r"^alpha[_0-9a-zA-Z.]+",T[j][0]) and T[j][1]=="O":
            T[j]=(T[j][0],'B:update')
        if re.search(r"^beta[_0-9a-zA-Z.]+",T[j][0]) and T[j][1]=="O":
            T[j]=(T[j][0],'B:update')
        if re.search(r"^pre[0-9a-zA-Z._-]+",T[j][0]) and T[j][1]=="O" and not re.search(r"^pre[a-z]+$",T[j][0]):
            T[j]=(T[j][0],'B:update')
        if re.search(r"^release[_\-a-zA-Z0-9]+",T[j][0]) and T[j][1]=="O":
            T[j]=(T[j][0],'B:update')	
        if re.search(r"^update[_\-a-zA-Z0-9]+",T[j][0]) and T[j][1]=="O" and not re.search(r"^update[a-z]+$",T[j][0]):
            T[j]=(T[j][0],'B:update')
        if re.search(r"\bbeta\b|\bBeta\b|\balpha\b|\bAlpha\b", T[j][0]):
            A=surroundings(T, j, 3)[0:3]
            for s,tag in A:
                if tag=="B:version":
                    T[j]=(T[j][0],'B:update')

        # cve_id_tagger 
        if re.search(r'CVE-[0-9]{4}-[0-9]{4}' , T[j][0]):
            T[j]=(T[j][0], "B:cve id")

        # cwe relevant term tagging
        if len(T)>j+2:
            for x,y,z in threes:
                if (T[j][0].lower(),T[j+1][0].lower(),T[j+2][0].lower())==(x.lower(),y.lower(),z.lower()):
                    if (T[j][1],T[j+1][1],T[j+2][1])==("O","O","O"):
                        T[j]=(T[j][0],"B:relevant_term")
                        T[j+1]=(T[j+1][0],"I:relevant_term")
                        T[j+2]=(T[j+2][0],"I:relevant_term")
        if len(T)>j+1:
            for x,y in twos:
                if (T[j][0].lower(),T[j+1][0].lower())==(x.lower(),y.lower()):
                    if (T[j][1],T[j+1][1])==("O","O"):
                        T[j]=(T[j][0],"B:relevant_term")
                        T[j+1]=(T[j+1][0],"I:relevant_term")
        for x in ones:
            if T[j][0].lower()==x[0].lower() and T[j][1]=="O":
                T[j]=(T[j][0],"B:relevant_term")

        # java terms tagger  
        if T[j][0]=="JavaScript":
            T[j]=(T[j][0],"B:programming language")
        if T[j][0].lower()=="java" and j!=0 and T[j-1][0].lower()=="oracle":
            T[j-1]=(T[j-1][0],"B:vendor")
            T[j]=(T[j][0],"B:application")
        if T[j][0].lower()=="java" and j!=0 and T[j-1][0].lower()=="sun":
            T[j-1]=(T[j-1][0],"B:vendor")
            T[j]=(T[j][0],"B:application")
        if T[j][0].lower()=="java" and len(T)>j+2 and T[j+1][0].lower()=="runtime" and T[j+2][0].lower()=="environment":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            T[j+2]=(T[j+2][0],"I:application")
            if len(T)>j+3 and re.search(r'^[0-9.\-_]+$',T[j+3][0]):
                T[j+3]=(T[j+3][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+2 and T[j+1][0].lower()=="web" and T[j+2][0].lower()=="start":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            T[j+2]=(T[j+2][0],"I:application")
            if len(T)>j+3 and re.search(r'^[0-9.\-_]+$',T[j+3][0]):
                T[j+3]=(T[j+3][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+1 and T[j+1][0].lower()=="plug-in":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            if len(T)>j+2 and re.search(r'^[0-9.\-_]+$',T[j+2][0]):
                T[j+2]=(T[j+2][0],"B:version")

        if T[j][0].lower()=="java" and len(T)>j+1 and T[j+1][0].lower()=="se":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            if len(T)>j+2 and re.search(r'^[0-9.\-_]+$',T[j+2][0]):
                T[j+2]=(T[j+2][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+1 and T[j+1][0].lower()=="ee":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            if len(T)>j+2 and re.search(r'^[0-9.\-_]+$',T[j+2][0]):
                T[j+2]=(T[j+2][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+1 and T[j+1][0].lower()=="me":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            if len(T)>j+2 and re.search(r'^[0-9.\-_]+$',T[j+2][0]):
                T[j+2]=(T[j+2][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+2 and T[j+1][0].lower()=="for" and T[j+2][0].lower()=="business":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            T[j+2]=(T[j+2][0],"I:application")
            if len(T)>j+3 and re.search(r'^[0-9.\-_]+$',T[j+3][0]):
                T[j+3]=(T[j+3][0],"B:version")
        if re.search(r'^j2se[0-9]*$' , T[j][0].lower()):
            T[j]=(T[j][0],"B:application")
        if T[j][0].lower()=="java" and len(T)>j+2 and T[j+1][0].lower()=="system" and T[j+2][0].lower()=="web":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            T[j+2]=(T[j+2][0],"I:application")
            if len(T)>j+3 and re.search(r'^[0-9.\-_]+$',T[j+3][0]):
                T[j+3]=(T[j+3][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+3 and T[j+1][0].lower()=="system" and T[j+2][0].lower()=="web" and T[j+3][0].lower()=="server":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            T[j+2]=(T[j+2][0],"I:application")
            T[j+3]=(T[j+3][0],"I:application")
            if len(T)>j+4 and re.search(r'^[0-9.\-_]+$',T[j+4][0]):
                T[j+4]=(T[j+4][0],"B:version")
        if T[j][0].lower()=="java" and len(T)>j+3 and T[j+1][0].lower()=="system" and T[j+2][0].lower()=="access" and T[j+3][0].lower()=="manager":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")
            T[j+2]=(T[j+2][0],"I:application")
            T[j+3]=(T[j+3][0],"I:application")
            if len(T)>j+4 and re.search(r'^[0-9.\-_]+$',T[j+4][0]):
                T[j+4]=(T[j+4][0],"B:version")
        if T[j][0]=="JavaVM":
            T[j]=(T[j][0],"B:application")
        if len(T)>j+1 and T[j+1][1]=="B:version" and T[j][0].lower()=="java":
            T[j]=(T[j][0],"B:application")
        if T[j][0].lower()=="java" and j!=0:
            if T[j-1][1]=="B:vendor" or T[j-1][1]=="I:vendor":
                T[j]=(T[j][0],"B:application")
        if len(T)>j+1 and T[j+1][0]!="" and re.search(r'[A-Z]',T[j+1][0][0]) and T[j][0].lower()=="java":
            T[j]=(T[j][0],"B:application")
            T[j+1]=(T[j+1][0],"I:application")

        # function, methods, and parameter tagging
        if re.search(r'^[a-zA-Z0-9\_\.]*[a-z0-9]+[A-Z]+',T[j][0]) or re.search(r'^[a-zA-Z0-9\_\.]*[A-Za-z0-9\.]+\_[a-zA-Z0-9\.]+'  ,  T[j][0]):
            if len(T)>j+1:
                if re.search(r'^function', T[j+1][0]):
                    T[j]=(T[j][0],"B:function")
                    if j>1 and re.search(r'(^and$|^or$)',T[j-1][0]) and re.search(r'(^[a-zA-Z0-9\_\.]*[a-z0-9]+[A-Z]+|^[a-zA-Z0-9\_\.]*[A-Za-z0-9\.]+\_[a-zA-Z0-9\.]+)'  ,  T[j-2][0]):
                        T[j-2]=(T[j-2][0],"B:function")
            if len(T)>j+1:
                if re.search(r'^method', T[j+1][0]):
                    T[j]=(T[j][0],"B:method")
                    if j>1 and re.search(r'(^and$|^or$)',T[j-1][0]) and re.search(r'(^[a-zA-Z0-9\_\.]*[a-z0-9]+[A-Z]+|^[a-zA-Z0-9\_\.]*[A-Za-z0-9\.]+\_[a-zA-Z0-9\.]+)'  ,  T[j-2][0]):
                        T[j-2]=(T[j-2][0],"B:method")
                if re.search(r'^parameter',T[j+1][0]):
                    T[j]=(T[j][0],"B:parameter")
                    if j>1 and re.search(r'(^and$|^or$)',T[j-1][0]) and re.search(r'(^[a-zA-Z0-9\_\.]*[a-z0-9]+[A-Z]+|^[a-zA-Z0-9\_\.]*[A-Za-z0-9\.]+\_[a-zA-Z0-9\.]+)'  ,  T[j-2][0]):
                        T[j-2]=(T[j-2][0],"B:parameter")

        # file tagging
        if re.search(r'\.[a-zA-Z0-9]{1,4}$',T[j][0]):
            if j>1 and re.search(r"^function",T[j-2][0]) and T[j-1][0]=="in":
                T[j]=(T[j][0],"B:file")	
            if len(T)>j+1 and re.search(r'^file$|^files$|^script$|^scripts$',T[j+1][0]):	
                T[j]=(T[j][0],"B:file")
        if re.search(r'[a-zA-Z0-9.\-_]+/[a-zA-Z0-9.\-_]+\.[a-zA-Z0-9]{0,4}$', T[j][0]):
            T[j]=(T[j][0],"B:file")

        # component and plug-in tagger
        if re.search("component", T[j][0]) or re.search("plugin", T[j][0]) :
            k=j-1
            i=0
            while k>0:
                if re.search(r"^[A-Z]+",T[k][0])and T[k][1]=="O":
                    T[k]=(T[k][0], "I:application")
                    k=k-1
                    i+=1
                else:
                    k=-1
            if i>0:
                T[j-i]=(T[j-i][0],"B:application")

        if j>1 and re.search("plug", T[j-2][0]) and re.search("-", T[j-1][0]) and re.search("in", T[j][0]):
            k=j-1
            i=0
            while k>0:
                if re.search(r"^[A-Z]+",T[k][0])and T[k][1]=="O":
                    T[k]=(T[k][0], "I:application")
                    k=k-1
                    i+=1
                else:
                    k=-1
            if i>0:
                T[j-i]=(T[j-i][0],"B:application")


        # ad hoc tagger
        if T[j][0]=="Windows" and j>0:
            T[j]=(T[j][0], "B:os")
        if T[j][0]=="Internet" and len(T)>j+1 and T[j+1][0]=="Explorer":
            T[j]=(T[j][0], "B:application")
            T[j+1]=(T[j+1][0], "I:application")
        if T[j][0]=="Apple" and len(T)>j+1 and T[j+1][0]=="Safari":
            T[j]=(T[j][0], "B:vendor")
            T[j+1]=(T[j+1][0], "B:application")
        if re.search("Oracle", T[j][0]):
            k=j+1
            i=0
            while k<len(T):
                if re.search(r"^[A-Z]+",T[k][0]) and T[k][1]=="O":
                    T[k]=(T[k][0], "I:application")
                    k=k+1
                    i+=1
                else:
                    k=len(T)
            if i>0:
                T[j]=(T[j][0], "B:vendor")
                T[j+1]=(T[j+1][0],"B:application")
        if T[j][0]=="WebKit":
            T[j]=(T[j][0], "B:application")

        # Items I added
        if T[j][0] == "Firefox" or T[j][0] =="Chrome" or T[j][0] =="IE":
            T[j]=(T[j][0], "B:application")
            if j>0 and T[j-1][0]=="Mozilla" or T[j-1][0]=="Google" or T[j-1][0]=="Microsoft":
                T[j-1]=(T[j-1][0], "B:vendor")
                
    return T

In [46]:
def surroundings(t,j,n):
    """ 
    t is a list, j is an index, n is the number of words 
    requested before and after t[j]
    """
    a=max(0,j-n)
    b=min(len(t), j+n)
    return t[a:b+1]

In [47]:
word_tag_pairs = [
    ('does', ''),
    ('this', ''),
    ('work', ''),
    ('on', ''),
    ('Firefox', ''),
    ('browser', ''),
    ('?', '')
]

In [48]:
print(secondary_tagger(word_tag_pairs))

[('does', ''), ('this', ''), ('work', ''), ('on', ''), ('Firefox', 'B:application'), ('browser', ''), ('?', '')]


In [49]:
print(secondary_tagger([('Firefox', '')]))

[('Firefox', 'B:application')]


# Appendix (spaCy tagging)

This code was used to tag the 11k tweets with spaCy NER. 

In [50]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [51]:
def tag_ner(sentence):
    doc = nlp(sentence)
    tupleList = []
    for token in doc:
        if token.ent_type_:
            tupleList.append((token, token.ent_iob_, token.ent_type_))
        else:
            tupleList.append((token, token.ent_iob_, ""))
    return tupleList

In [52]:
df = pd.read_csv('tweetTextCheck.csv',sep=',', engine='python')
df = df[['TweetText']]
df['TweetText'] = df['TweetText'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))
df.head()

,TweetText
0,"i appreciate the love for hacking guys, but le..."
1,@i_oofficial is such a great hacker he made hi...
2,Cybersecurity Experts Point to Iran for Bitcoi...
3,Domain auction \nDomain name: https://t.co/zZd...
4,#Depressed #Security Researcher Drops Serious ...


In [53]:
df['ner_labels'] = df['TweetText'].apply(tag_ner)
df.head()

,TweetText,ner_labels
0,"i appreciate the love for hacking guys, but le...","[(i, O, ), (appreciate, O, ), (the, O, ), (lov..."
1,@i_oofficial is such a great hacker he made hi...,"[(@i_oofficial, O, ), (is, O, ), (such, O, ), ..."
2,Cybersecurity Experts Point to Iran for Bitcoi...,"[(Cybersecurity, B, ORG), (Experts, I, ORG), (..."
3,Domain auction \nDomain name: https://t.co/zZd...,"[(Domain, O, ), (auction, O, ), (\n, O, ), (Do..."
4,#Depressed #Security Researcher Drops Serious ...,"[(#, B, MONEY), (Depressed, I, MONEY), (#, I, ..."


In [54]:
df['tokenized'] = df['TweetText'].apply(lambda x: nltk.word_tokenize(x))
df.head()

,TweetText,ner_labels,tokenized
0,"i appreciate the love for hacking guys, but le...","[(i, O, ), (appreciate, O, ), (the, O, ), (lov...","[i, appreciate, the, love, for, hacking, guys,..."
1,@i_oofficial is such a great hacker he made hi...,"[(@i_oofficial, O, ), (is, O, ), (such, O, ), ...","[@, i_oofficial, is, such, a, great, hacker, h..."
2,Cybersecurity Experts Point to Iran for Bitcoi...,"[(Cybersecurity, B, ORG), (Experts, I, ORG), (...","[Cybersecurity, Experts, Point, to, Iran, for,..."
3,Domain auction \nDomain name: https://t.co/zZd...,"[(Domain, O, ), (auction, O, ), (\n, O, ), (Do...","[Domain, auction, Domain, name, :, https, :, /..."
4,#Depressed #Security Researcher Drops Serious ...,"[(#, B, MONEY), (Depressed, I, MONEY), (#, I, ...","[#, Depressed, #, Security, Researcher, Drops,..."


In [55]:
rowsList = []

rowsList.append(['tweetNum', 'word', 'ner_iob', 'ner_tag'])

curSentence = 0
for i, row in df.iterrows():
    # create row for each word in tweet
    for token, ner_labels in zip(row['tokenized'], row['ner_labels']):
        curTokenRow = []
        curTokenRow.append(curSentence)
        curTokenRow.append(token)
        curTokenRow.append(ner_labels[1])
        curTokenRow.append(ner_labels[2])
        rowsList.append(curTokenRow)
    curSentence = curSentence + 1

In [56]:
headers = rowsList.pop(0)

dfFormatted = pd.DataFrame(rowsList, columns=headers)
dfFormatted.head(50)

,tweetNum,word,ner_iob,ner_tag
0,0,i,O,
1,0,appreciate,O,
2,0,the,O,
3,0,love,O,
4,0,for,O,
5,0,hacking,O,
6,0,guys,O,
7,0,",",O,
8,0,but,O,
9,0,let,O,


In [57]:
dfFormatted.tail(50)

,tweetNum,word,ner_iob,ner_tag
5486,298,on,O,
5487,298,Windows,O,
5488,298,10,O,
5489,298,:,O,
5490,298,https,O,
5491,298,:,O,
5492,298,//t.co/Sk5epLIvOd,O,
5493,298,#,O,
5494,298,exploitation,O,
5495,298,#,O,


# Import dataset

In [7]:
# df = pd.read_csv('tweetTextCheck.csv',sep=',', engine='python')
# df.head()

,Unnamed: 0,TweetText,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,0,"i appreciate the love for hacking guys, but le...",NaN,NaN,NaN,NaN
1,1,@i_oofficial is such a great hacker he made hi...,NaN,NaN,NaN,NaN
2,2,Cybersecurity Experts Point to Iran for Bitcoi...,NaN,NaN,NaN,NaN
3,3,💰Domain auction ⏳\nDomain name: https://t.co/z...,NaN,NaN,NaN,NaN
4,4,#Depressed #Security Researcher Drops Serious ...,NaN,NaN,NaN,NaN


## data transformation

In [8]:
# df = df[['TweetText']]

In [9]:
# df.head()

,TweetText
0,"i appreciate the love for hacking guys, but le..."
1,@i_oofficial is such a great hacker he made hi...
2,Cybersecurity Experts Point to Iran for Bitcoi...
3,💰Domain auction ⏳\nDomain name: https://t.co/z...
4,#Depressed #Security Researcher Drops Serious ...


In [10]:
# # remove emojis
# df['TweetText'] = df['TweetText'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))

In [11]:
# # tokenize text
# df['tokenized'] = df['TweetText'].apply(lambda x: nltk.word_tokenize(x))
# df.head()

,TweetText,tokenized
0,"i appreciate the love for hacking guys, but le...","[i, appreciate, the, love, for, hacking, guys,..."
1,@i_oofficial is such a great hacker he made hi...,"[@, i_oofficial, is, such, a, great, hacker, h..."
2,Cybersecurity Experts Point to Iran for Bitcoi...,"[Cybersecurity, Experts, Point, to, Iran, for,..."
3,Domain auction \nDomain name: https://t.co/zZd...,"[Domain, auction, Domain, name, :, https, :, /..."
4,#Depressed #Security Researcher Drops Serious ...,"[#, Depressed, #, Security, Researcher, Drops,..."


In [12]:
# rowsList = []

# rowsList.append(['tweetNum', 'word'])

# curSentence = 1
# for i, row in df.iterrows():
#     # create row for each word in tweet
#     for token in row['tokenized']:
#         curTokenRow = []
#         curTokenRow.append(curSentence)
#         curTokenRow.append(token)
#         rowsList.append(curTokenRow)
#     curSentence = curSentence + 1

In [13]:
# headers = rowsList.pop(0)

In [14]:
# dfFormatted = pd.DataFrame(rowsList, columns=headers)
# dfFormatted.head()

,tweetNum,word
0,1,i
1,1,appreciate
2,1,the
3,1,love
4,1,for


# Apply regex tagger

In [58]:
def tag_single_token(tok):
    formatted_tok = []
    formatted_tok.append((tok, ""))
    
    tagged = secondary_tagger(formatted_tok)
    if (tagged[0][1] == ''):
        return "None"
    return tagged[0][1]

In [59]:
tag_single_token("Ransomware")

'None'

In [60]:
dfFormatted['tag'] = dfFormatted['word'].apply(lambda x: tag_single_token(x))
dfFormatted.tail(10)

,tweetNum,word,ner_iob,ner_tag,tag
5526,301,Anti,O,,None
5527,301,DDoS,O,,None
5528,301,Attack,O,,None
5529,301,#,O,,None
5530,301,Prestashop,O,,None
5531,301,https,O,,None
5532,301,:,O,,None
5533,301,//t.co/xUGxiI8zw8,O,,None
5534,301,https,O,,None
5535,301,:,O,,None


In [61]:
dfFormatted.to_csv('300Tweet_autoLabel_rule_V2.csv', sep=',')

In [62]:
dfFormatted[dfFormatted['ner_tag'] != "None"]

,tweetNum,word,ner_iob,ner_tag,tag
0,0,i,O,,None
1,0,appreciate,O,,None
2,0,the,O,,None
3,0,love,O,,None
4,0,for,O,,None
...,...,...,...,...,...
5531,301,https,O,,None
5532,301,:,O,,None
5533,301,//t.co/xUGxiI8zw8,O,,None
5534,301,https,O,,None
